# Asko analysis - the R part

- Sample clustering
- Spacer profiles
- Sample correlation vs Spatial correlation

In [1]:
# install.packages(c("PoiClaClu", "cluster", "vegan", "ape", "lubridate", "stringi", "pheatmap"))

library(cluster)
library(vegan)
library(ape)
library("PoiClaClu")
library(lubridate)
library(stringi)
library(pheatmap)
library(calibrate)

# laptop config
setwd("/home/sergiu/data/work/andersson/src/andersson/src")
crispr_tab_location = "/home/sergiu/data/work/andersson/data/asko/spacer_table_filtered_all.txt"

# desktop config
#setwd("/home/sergiu/data/data/work/andersson/src/andersson/src")
#spacerfile <- "/home/sergiu/data/data/work/andersson/data/asko/spacer_table_filtered.txt"

samples_r1 <- c('SRR3727523_R1', 'SRR3727518_R1', 'SRR3727515_R1', 'SRR3727513_R1', 'SRR3727509_R1', 'SRR3727505_R1')
samples_r2 <- c('SRR3727523_R2', 'SRR3727518_R2', 'SRR3727515_R2', 'SRR3727513_R2', 'SRR3727509_R2', 'SRR3727505_R2')

Loading required package: permute
Loading required package: lattice
This is vegan 2.4-4

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: MASS

Attaching package: ‘calibrate’

The following objects are masked from ‘package:vegan’:

    calibrate, rda



In [2]:
# TEMP: R test
source("analize_asko.R")
mult(2,3)

[1] 6

In [3]:

tab <- read.delim(spacerfile)
crispr_id <- as.character(tab[,1])
crispr_repeat_seq <- as.character(tab[,2])
spacer_id <- as.character(tab[,3])
spacer_repeat_seq <- as.character(tab[,4])
counts <- as.matrix(tab[,5:ncol(tab)])
colnames(counts) = gsub("^X", "", colnames(counts))
sample = colnames(counts)

norm_counts <- counts
for (i in 1:ncol(counts)) {
  norm_counts[,i] <- counts[,i]/sum(counts[,i])
}

binary_counts <- counts
binary_counts[which(counts > 0)] <- 1

nonred_crispr_id = unique(crispr_id)
counts_per_crispr = matrix(ncol = length(sample), nrow = length(nonred_crispr_id))
colnames(counts_per_crispr) = sample
rownames(counts_per_crispr) = nonred_crispr_id
for (i in 1:length(nonred_crispr_id)) {
  ix = which(crispr_id==nonred_crispr_id[i])
  if (length(ix) > 1) {
    counts_per_crispr[i,] = apply(counts[ix,], 2, sum)
  } else {
    counts_per_crispr[i,] = counts[ix,]
  }
}

norm_counts_per_crispr <- counts_per_crispr
for (i in 1:ncol(counts_per_crispr)) {
  norm_counts_per_crispr[,i] <- counts_per_crispr[,i]/sum(counts_per_crispr[,i])
}

binary_counts_per_crispr <- counts_per_crispr
binary_counts_per_crispr[which(counts_per_crispr > 0)] <- 1

nonred_spacers_per_crispr = c()
for (i in 1:length(nonred_crispr_id)) {
  ix = which(crispr_id==nonred_crispr_id[i])
  nonred_spacers_per_crispr[i] = length(ix)
}


ERROR: Error in read.table(file = file, header = header, sep = sep, quote = quote, : object 'spacerfile' not found


# Sample pairwise distances

In [ ]:
binary_dist <- as.matrix(dist(t(counts), method = "binary"))
euclidean_dist <- as.matrix(dist(t(norm_counts), method = "euclidean"))
poison_dist = as.matrix(PoissonDistance(t(counts))$dd)
rownames(poison_dist) = colnames(poison_dist) = sample

binary_dist_per_crispr <- as.matrix(dist(t(counts_per_crispr), method = "binary"))
euclidean_dist_per_crispr <- as.matrix(dist(t(norm_counts_per_crispr), method = "euclidean"))
poison_dist_per_crispr = as.matrix(PoissonDistance(t(counts_per_crispr))$dd)
rownames(poison_dist_per_crispr) = colnames(poison_dist_per_crispr) = sample

More pairwise distances were computed above, but we only look at the Poisson distance as it seems the most informative.

In [ ]:
pheatmap(poison_dist_per_crispr[samples_r1,samples_r1], cluster_rows = TRUE, cluster_cols = TRUE)

In [ ]:
pcoa <- pcoa(poison_dist_per_crispr[samples_r1, samples_r1], correction = "cailliez")
xlab = paste("PC1 (", 100*round(pcoa$values[1,3], 2), "%)", sep = "")
ylab = paste("PC2 (", 100*round(pcoa$values[2,3], 2), "%)", sep = "")
plot(pcoa$vectors[,1], pcoa$vectors[,2], pch = 16, col = "red", xlab = xlab, ylab = ylab)
textxy(pcoa$vectors[,1], pcoa$vectors[,2], samples_r1, cex = 0.5, offset = 0.6)

Anders also used a package from community ecology ([vegan](https://cran.r-project.org/web/packages/vegan/)) to compute sample distance.


In [ ]:
mds <- metaMDS(poison_dist_per_crispr)

In [ ]:
plot(mds, type = "t")

# Sample clustering

In [ ]:
cluster_sample <- agnes(poison_dist, diss = TRUE, method = "average")
plot(cluster_sample, which.plots = 2, hang = 0, main = "", axes = FALSE, xlab = "", ylab = "", sub = "", cex = 1.0)